In [1]:
import sys
import os
from pathlib import Path
curr_dir = os.path.abspath('')
project_dir = Path(curr_dir).parent
sys.path.append(f"{project_dir}")
sys.path.append(f"{project_dir}/src")
sys.path

['/Users/albertinopadin/Desktop/Dev/Python Projects/PlaneRecognizer/notebooks',
 '/Users/albertinopadin/miniforge3/envs/dev/lib/python39.zip',
 '/Users/albertinopadin/miniforge3/envs/dev/lib/python3.9',
 '/Users/albertinopadin/miniforge3/envs/dev/lib/python3.9/lib-dynload',
 '',
 '/Users/albertinopadin/miniforge3/envs/dev/lib/python3.9/site-packages',
 '/Users/albertinopadin/miniforge3/envs/dev/lib/python3.9/site-packages/IPython/extensions',
 '/Users/albertinopadin/.ipython',
 '/Users/albertinopadin/Desktop/Dev/Python Projects/PlaneRecognizer',
 '/Users/albertinopadin/Desktop/Dev/Python Projects/PlaneRecognizer/src']

In [1]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [2]:
from ImageObjectDetectors.TensorflowCNN4Images import TensorflowCNN4Images
from JetRecognizer.JetRecognizerPreprocessing import load_label_encoder, normalize_pixels_in_img_obj, convert_labels_to_one_hot_vectors, \
    get_random_validation_fighter_images_as_pixel_values_generator, get_random_test_fighter_images_as_pixel_values_generator
from time import perf_counter
from ImageObjectDetectors.CNN4ImagesBase import KernelProgression
from sklearn.metrics import precision_score, accuracy_score
from Common.DL_FilePaths import IMG_SIZE
import ipywidgets as widgets
from fastai.vision.all import *
# from PIL import Image

/Users/albertinopadin/miniforge3/envs/dev/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


physical_devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
INPUT_SHAPE = (960, 960, 3)
N_OUTPUT = 6
LEARNING_RATE = 0.001

# For Mac-trained model:
JET_RECOGNIZER_MODEL_FILENAME = 'jet_recognizer_apple_silicon' + '_' + str(INPUT_SHAPE[0])

# For Linux-trained model:
# JET_RECOGNIZER_MODEL_FILENAME = 'jet_recognizer_A6000' + '_' + str(INPUT_SHAPE[0])


LABEL_ENCODER_FILENAME = "jet_label_classes.npy"

# Still need to define the model, might want to refactor so don't have to...
jet_recognizer = TensorflowCNN4Images(INPUT_SHAPE,
                                      N_OUTPUT,
                                      LEARNING_RATE,
                                      kernel_progression=KernelProgression.KERNEL_GETS_BIGGER)

jet_recognizer.load_model(JET_RECOGNIZER_MODEL_FILENAME)
label_encoder = load_label_encoder(LABEL_ENCODER_FILENAME)

Using Kernel Progression: KernelProgression.KERNEL_GETS_BIGGER
TF Keras backend: <module 'keras.api._v2.keras.backend' from '/Users/albertinopadin/miniforge3/envs/dev/lib/python3.9/site-packages/keras/api/_v2/keras/backend/__init__.py'>
Metal device set to: Apple M1 Max
Input Shape: (960, 960, 3)
Model: "ImageCNN_TF"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv64_input_layer (Conv2D)  (None, 480, 480, 64)     4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 240, 240, 64)     0         
 )                                                               
                                                                 
 conv64_k5_layer (Conv2D)    (None, 120, 120, 64)      102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                 

In [4]:
def preprocess_img(img, to_size=IMG_SIZE.IS_960):
    img.thumbnail((to_size.value, to_size.value))
    return pad_pil_image(img, to_size)

In [5]:
btn_upload = widgets.FileUpload()

out_pl = widgets.Output()
out_pl.clear_output()

lbl_pred = widgets.Label()

btn_run = widgets.Button(description='Classify')

In [6]:
# Adapted from fastai book p. 82:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    # img = Image.open(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(128,128))
    # print(f'img shape: {img.shape}')
    normalized_img = normalize_pixels_in_img_obj(img)
    # print(f'normalized_img shape: {normalized_img.shape}')
    pred = jet_recognizer.predict(np.array([normalized_img]), flatten_output=False, one_hot=True)
    print(f'pred: {pred}')
    pred_label = label_encoder.inverse_transform(np.argmax(pred, axis=1))[0]
    print(f'Pred label: {pred_label}')
    lbl_pred.value = f'Prediction: {pred_label}'

In [7]:
btn_run.on_click(on_click_classify)

In [8]:
widgets.VBox([
    widgets.Label('Upload jet image'),
    btn_upload,
    btn_run,
    out_pl,
    lbl_pred
])